In [1]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM,Dense,Input,ConvLSTM2D,Reshape,Activation,Lambda,Softmax,Concatenate
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout,RepeatVector,Reshape,Activation,Lambda,Softmax
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import re
import pandas as pd
from keras import backend as K
import keras.backend as kerback
#from keras.models import Model
from tensorflow.keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow as tf
from keras.optimizers import Adam

In [2]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy import random
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import UpSampling2D, Reshape, Activation, Conv2D, BatchNormalization, LeakyReLU, Input, Flatten, multiply,Conv2DTranspose
from keras.layers import Dense, Embedding
from keras.models import Sequential, Model

In [3]:
def prepare_data(f):
    data = pd.read_csv(f, sep="\n",header=None)
    list_listword=[]#liste des listes des mots de chaque tweet
    list_tweets=data[0].values.tolist()#liste des tweets
    l=[]
    for text in list_tweets:
        text = re.sub(r'http\S+', '', text)   # Remove URLs
        text = re.sub(r'—', ' ', text) 
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
        text = text.strip(" ")   # Remove whitespace resulting from above
        text = re.sub(r' +', ' ', text)   # Remove redundant spaces
        l.append(text)
    
    return l

In [4]:
list_tweets=prepare_data('C:/tweet.txt')

In [5]:
def tokenizer_data(list_tweets):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list_tweets)
    sequences = tokenizer.texts_to_sequences(list_tweets)
    vocab_size=len(tokenizer.word_index)
    return sequences, tokenizer, vocab_size

def max_tweet(sequences):
    max_length=0
    for i in range(len(sequences)):
        if(len(sequences[i])>max_length):
            max_length=len(sequences[i])
    return max_length

sequences, tokenizer, vocab_size=tokenizer_data(list_tweets)
max_length=max_tweet(sequences)
sequences=np.array([xi+[0]*(max_length-len(xi)) for xi in sequences])
y=sequences[:,-1]
print(y.shape)
y_train = y.reshape(-1, 1)
X=sequences
X_train = np.expand_dims(X, axis=2)
nb_tweet=len(sequences)
batch_size=max_length
print(vocab_size)

(66,)
841
(66,)
841


In [6]:
import keras.backend as kerback
# define the standalone generator model
def build_generator(n_outputs):#max_length nombre d'element das une tweet
   
    noise=Input(shape=(57,1))
    print(noise.shape)
    label=Input(shape=(57,1))
    print(label.shape)
    #label_emb=Embedding(vocab_size,1)(label)
    
    x0=Concatenate(axis=2)([noise,label])
    x1=LSTM(15, activation='relu',input_dim=n_outputs)(x0)
    #15 nombre de neurone ,batch_size=n_batch=20 
    # repeat vector
    x2=RepeatVector(n_outputs)(x1)
    # decoder layer
    x3=LSTM(15, activation='relu', return_sequences=True)(x2)
    x4=TimeDistributed(Dense(1))(x3)
    unstacked = Lambda(lambda x: tf.unstack(x, axis=2))(x4)
    dense_outputs = [Dense(n_outputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)

    #generated_tweet=model(x0)
    model = Model([noise,label],merged)
    model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    print('generator')
    print(model.summary())
    return  model

build_generator(57)

(None, 57, 1)
(None, 57, 1)
generator
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 57, 2)        0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 15)       

In [7]:
# define the standalone discriminator model
def build_discriminator(n_inputs):
    tweet=Input(shape=(n_inputs,1)) #n_imput nombre des element 
    
    label=Input(shape=(n_inputs,1))
    #label_emb=Embedding(vocab_size,1)(label)
    
    x0=Concatenate(axis=1)([tweet,label])
    x2=LSTM(15, activation='relu', input_dim=n_inputs)(x0)
    x3=RepeatVector(n_inputs)(x2)
    x4=LSTM(15, activation='relu', return_sequences=True)(x3)
    x5=TimeDistributed(Dense(1))(x4)
    unstacked = Lambda(lambda x: tf.unstack(x, axis=2))(x5)
    dense_outputs = [Dense(n_inputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)
    x1=Flatten()(merged)
    x6=Dense(1, activation='sigmoid')(x1)
    model = Model([tweet,label], x6)
    print(model.summary())
    model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])
    return model
build_discriminator(57)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 114, 1)       0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 15)           1020        concatenate_1[0][0]   

In [8]:
# define the combined generator and discriminator model
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    #model.add(generator)
    x0=Input(shape=(57,1))
    x1=Input(shape=(57,))
    x2=generator([x0,x1])
    print(x2[-1].shape)
    x3=discriminator([x2,x1])
    print(x3.shape)
    model = Model([x0,x1], x3)
    # add the discriminator
    #model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    print(model.summary())
    return model
define_gan(build_generator(57), build_discriminator(57))

(None, 57, 1)
(None, 57, 1)
(None, 57, 1)
(None, 57, 1)
generator
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 57, 2)        0           input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)        

In [10]:
# Create batch labels to use when calling train_on_batch
valid = ones((max_length,1))
fake = zeros((max_length,1))

In [11]:
generator = build_generator(max_length)
# Build and compile the discriminator
discriminator = build_discriminator(max_length)
#discriminator.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])

# Build and compile the combined model
gan_model = define_gan(generator, discriminator)

(None, 57, 1)
(None, 57, 1)
(None, 57, 1)
(None, 57, 1)
generator
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 57, 2)        0           input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
lstm_8 (LSTM)       

In [28]:
def sample_tweet(epoch,noise,sampled_labels,g_model):
    result = g_model.predict([noise,sampled_labels])
    #print(result)
    #print(result.shape)
    #print(len(result))
    res=[]
    predicted_word =""
    for j in range(57):
        #predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == abs(int(result[0][j])):
                predicted_word = predicted_word+' '+word
                break
    res.append(predicted_word)
    print(epoch,res)
    
    #for i in res:
    #    print(i)
    return res

In [29]:
def disc_text(model_disc, seed_text):
    #print(seed_text)
    sequences, tokenizer=tokenizer_data(seed_text)
    sequences= pad_sequences(sequences, maxlen=max_length, padding='post')
    y = model_disc.predict(sequences)
    #print(y)
    #for i in y:
    if(i<0.7):
        print("fake")
    else:
        print("real")

In [31]:
# Store the losses
d_losses = []
g_losses = []
sample_period = 100 # every `sample_period` steps generate and save some data"
Epoch=10
#x_real= generate_real_samples()
X =sequences[:33]
X = pad_sequences(X, maxlen=max_length, padding='post')
#print("X=",X)
y =sequences[33:]
y = pad_sequences(y, maxlen=max_length, padding='post')
#print("y=",y)
X_train=X.reshape(33, max_length, 1)
y_train=y.reshape(33, max_length, 1)
#y_train = y.reshape(-1, 1)
#print(X_train.shape)
#print(y_train.shape)
#X_train = np.expand_dims(X_train, axis=3)
#y_train = np.expand_dims(y_train, axis=3)
real = np.ones(shape=(33, 1))
fake = np.zeros(shape=(33, 1))
######################## Main training loop###########################
for i in range(Epoch):
        ########################
        ###Train Discriminator##
        #######################
        # prepare real samples
        [tweets, labels]= X_train,y_train
        #print(X_train.shape)
        #idx = np.random.randint(0, X_train.shape[0], batch_size)
        #print(idx.shape)
        
        #tweets, labels = X_train[idx], y_train[idx]
        #print(tweets.shape)
        #print(labels.shape)
        #print(labels)
        # Sample noise as generator input
        lab=labels[-1].reshape(1,57, 1)
        noise = np.random.randint(vocab_size, size=(1, 57,1))
        #print(noise.shape)
        # Generate fake tweets
        gen_twt= generator.predict([noise,lab])
        f=fake[-1].reshape(1, 1)
        
        # Train the discriminator
        # both loss and accuracy are returned
        d_loss_real, d_acc_real=discriminator.train_on_batch([tweets,labels], real)
        d_loss_fake, d_acc_fake =discriminator.train_on_batch([gen_twt,lab], f)
        d_loss = 0.5 * (d_loss_real + d_loss_fake)
        d_acc  = 0.5 * (d_acc_real + d_acc_fake)
        
        #######################
        ### Train generator ###
        #######################
    
        # Condition on labels
        sampled_labels = np.random.randint(0, 10, batch_size)                          
        #print(labels.shape) 
        noise = np.random.randint(vocab_size, size=(33, 57,1))
        y_gan = ones((33, 1))
        #print(y_gan.shape)
        lab=labels[random.randint(0, 33)].reshape(1,57, 1)
        # update the generator via the discriminator's error
        #noise=random.randn(57,1)
        #gan_model([noise,x_real[-1]], y_gan).summary()
        g_loss=gan_model.train_on_batch([noise,labels], y_gan)
        noise = np.random.randint(vocab_size, size=(1, 57,1))
        # Save the losses
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
        #if i % 2 == 100:#after 100 epoch we save
        #if i % sample_period == 0:
        #print(f"epoch: {i+1}, d_loss: {d_loss:.2f}, \d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
        print("d_loss",d_loss)
        print("d_acc:",d_acc)
        print("g_loss",g_loss)
        sample_tweet(i,noise,lab, generator)
        #disc_text(discriminator, tex)

d_loss 0.2404712811112404
d_acc: 0.9848484992980957
g_loss [0.44721269607543945, 0.9696969985961914]
0 [' voted movies thing snapshot dropped kept 866 other stole veterans preexisting else surely songs polls does ticket events stepped luther are threatening court states stepped reform rejoining in divisions let stake i’m office hank']
d_loss 0.2404712811112404
d_acc: 0.9848484992980957
g_loss [0.44721269607543945, 0.9696969985961914]
0 [' voted movies thing snapshot dropped kept 866 other stole veterans preexisting else surely songs polls does ticket events stepped luther are threatening court states stepped reform rejoining in divisions let stake i’m office hank']
d_loss 0.23936279211193323
d_acc: 0.9848484992980957
g_loss [0.4277613162994385, 0.9696969985961914]
1 [' election a that day our before day your georgia it today work is my our is the today you in i election have or at a do today our the you can my was that for right the election on this one all work vote with with my our m